**SLM MODEL FOR QUESTION ANSWERING**



Importing packages

In [7]:
import torch
import faiss
import nltk
from nltk.tokenize import sent_tokenize
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import warnings

Suppressing Warnings and Downloading NLTK Data

In [8]:
warnings.filterwarnings("ignore", category=UserWarning, module="nltk")
warnings.filterwarnings("ignore", category=UserWarning, module="transformers")

nltk.download('punkt', quiet=True)

True

Building a Slim Question-Answering Model

In [11]:
class SmallLanguageModel:
    def __init__(self, book_text):
        self.text = book_text
        self.sentences = sent_tokenize(self.text)
        self.embed_model = "sentence-transformers/all-MiniLM-L6-v2"
        self.sentence_model = SentenceTransformer(self.embed_model)

        self.retriever = self._build_index()

        self.qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2")

    def _build_index(self):
        sentence_embeddings = self.sentence_model.encode(self.sentences, convert_to_tensor=True)
        index = faiss.IndexFlatL2(sentence_embeddings.shape[1])
        index.add(sentence_embeddings.cpu().numpy())
        return index

    def retrieve_context(self, question, top_k=3):
        query_embedding = self.sentence_model.encode([question], convert_to_tensor=True)
        _, indices = self.retriever.search(query_embedding.cpu().numpy(), top_k)
        return " ".join([self.sentences[i] for i in indices[0]])

    def answer_question(self, question):
        context = self.retrieve_context(question)
        result = self.qa_model(question=question, context=context)
        return result["answer"], context

Interactive Q&A with a Small Language Model

In [12]:
book_path = '/content/Sample.txt'

with open(book_path, "r", encoding="utf-8") as f:
    book_text = f.read()

model = SmallLanguageModel(book_text)

question = input("Enter your question: ")

answer, context = model.answer_question(question)

print(f"\n📌 Question: {question}")
print(f"✅ Answer: {answer}")
print(f"📖 Context: {context}")

Device set to use cpu


Enter your question: Where did Alice ran across ?

📌 Question: Where did Alice ran across ?
✅ Answer: the field
📖 Context: There was nothing very remarkable in that; nor did Alice think it so very much out of the way to hear the Rabbit say to itself, 'Oh dear! (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural); but when the Rabbit actually took a watch out of its waistcoat pocket and looked at it and then hurried on, Alice started to her feet, for it flashed across her mind that she had never before seen a rabbit with either a waistcoat pocket, or a watch to take out of it, and burning with curiosity, she ran across the field after it, and fortunately was just in time to see it pop down a large rabbit-hole under the hedge. Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but